# TP 3: Hasting-Metropolis (and Gibbs) samplers

In [2]:
# imports 
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import invwishart
from tqdm import tqdm
import scipy

## Exercise 1: Hasting-Metropolis within Gibbs – Stochastic Approximation EM

#### Question 2 : Generate synthetic data from the model by taking some reasonable values for the parameters.

In [18]:
# First we fix some parameters 
N = 10 ** 3     # Number of individuals 
K = np.random.uniform(low = 8, high = 10, size = N)    # Number of observations per individual
K = K.astype(int)

# Note that sigma will always refers to the covariance matrix of the data (or the variance of the data) that's to say sigma = sigma^2 ! 
p0 = 1 
sigma_t0 = 1
sigma_v0 = 2.5 
t0_barbar, st0 = 1, 1
v0_barbar, sv0 = 2.5, 5
v_xi, m_xi = 0.5, 1
v_tau, m_tau = 0.5, 1
v, m = 0.5, 1

# We generate the data

def average_trajectory(t, p0, v0):
    """ Average trajectory """
    return p0 + v0 * (t - v0)

def individual_trajectory(t, p0, v0, t0, alpha_i, tau_i):
    """ Individual trajectory of an individual """
    return average_trajectory(alpha_i * (t - t0 - tau_i) + t0, p0, v0)[0]

def generate_data(N, K, p0, sigma_t0, sigma_v0, t0_barbar, st0, v0_barbar, sv0):
    """ 
    Parameters
    ----------
    
    Returns
    -------
    Y : np.array of size (N, K)
        Where K = max(K_i) for i in range(N)
        The data generated
    Y[i, :] denotes each individual i,  each individual has K_i observations Y[i, j] for j in range(K_i) and then 0 for j > K_i
    """
    Y = []
    sigma = invwishart.rvs(df = 1, scale = 1, size = N)
    sigma_xi = invwishart.rvs(df = 1, scale = 1, size = N)
    sigma_tau = invwishart.rvs(df = 1, scale = 1, size = N)
    t0_bar = np.random.normal(loc = t0_barbar, scale = st0, size = N)
    v0_bar = np.random.normal(loc = v0_barbar, scale = sv0, size = N)
    K_max = max(K)
    for i in range(N):
        Y_i = []
        K_i = K[i]
        epsilon_i = np.random.normal(loc = 0, scale = sigma[i], size = K_i)
        xi = np.random.normal(loc = 0, scale = sigma_xi[i], size = 1)
        alpha_i = np.exp(xi)
        tau_i = np.random.normal(loc = 0, scale = sigma_tau[i], size = 1)
        t_0 = np.random.normal(loc = t0_bar[i], scale = sigma_t0)
        v_0 = np.random.normal(loc = v0_bar[i], scale = sigma_v0)
        for j in range(1, K_i + 1):
            Y_ij = individual_trajectory(j, p0, v_0, t_0, alpha_i, tau_i) + epsilon_i[j - 1]
            Y_i.append(Y_ij)
        Y_i += [0] * (K_max - K_i)
        Y_i = np.array(Y_i)
        Y.append(Y_i)
    return np.array(Y)

Y = generate_data(N, K, p0, sigma_t0, sigma_v0, t0_barbar, st0, v0_barbar, sv0)
print("Y shape is :", Y.shape)

Y shape is : (1000, 9)


/var/folders/t6/khg6jj9s4_b0lqrt24p4bvs00000gn/T/ipykernel_2387/3065088622.py:50: RuntimeWarning: overflow encountered in exp
  alpha_i = np.exp(xi)
